# Train sentence by BERT features

## Deploy BERT server

Instruction website: https://bert-as-service.readthedocs.io/en/latest/section/get-start.html  
Download server and client:
``` bash
pip install -U bert-serving-server bert-serving-client  
```
Downlaod and unzip pretrained bert model(BERT-Large, Uncased, 1024 dimensional output):  
``` bash
cd ${model_path}
wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
unzip uncased_L-24_H-1024_A-16.zip  
```  

Start bert server at local machine: 
``` bash
bert-serving-start -model_dir ${model_path}/uncased_L-24_H-1024_A-16 -max_seq_len=100 -num_worker=1  
bert-serving-start -model_dir /share/ShareFolder/uncased_L-24_H-1024_A-16/ -max_seq_len=150 -gpu_memory_fraction=0.9 -num_worker=1
```
Then, call from client end in python:
``` python
from bert_serving.client import BertClient
bc = BertClient()
bc.encode(['First do it', 'then do it right', 'then do it better'])
```


## Load data as Pandas dataframe

In [2]:
import numpy as np
import pandas as pd

train_file_path = "./training.csv"
test_file_path = './dev.csv'


train_df = pd.read_csv(train_file_path)
# test_df = pd.read_csv(test_file_path)


train_df

KeyboardInterrupt: 

## Feature extraction

### Construct and save bert features to file for reuse

In [ ]:
from bert_serving.client import BertClient
bc = BertClient()

# for cased restart server with 
# bert-serving-start -model_dir /share/ShareFolder/cased_L-24_H-1024_A-16/ -max_seq_len=50 -cased_tokenization -gpu_memory_fraction=0.9 -num_worker=11
# for uncased restart server with 
#bert-serving-start -model_dir /share/ShareFolder/cased_L-24_H-1024_A-16/ -max_seq_len=50 -gpu_memory_fraction=0.9 -num_worker=1
case = "_cased"


In [10]:
# # train, test claim encode
# restart server with 
# bert-serving-start -model_dir /share/ShareFolder/uncased_L-24_H-1024_A-16/ -max_seq_len=50 -gpu_memory_fraction=0.9 -num_worker=1


train_claim_encode = bc.encode(list(train_df['claim']))
np.save("./Sentence_encodings/train_claim_encode" + case, train_claim_encode)

# test_claim_encode = bc.encode(list(test_df['claim']))
# np.save("./Sentence_encodings/test_claim_encode" + case, test_claim_encode)

In [11]:
# train, test evidence encode
# restart server with 
# bert-serving-start -model_dir /share/ShareFolder/uncased_L-24_H-1024_A-16/ -max_seq_len=150 -gpu_memory_fraction=0.9 -num_worker=1

train_evi_encode = bc.encode(list(train_df['evi_text']))
np.save("./Sentence_encodings/train_evi_encode" + case, train_evi_encode)

# test_evi_encode = bc.encode(list(test_df['evi_text']))
# np.save("./Sentence_encodings/test_evi_encode" + case, test_evi_encode)



/usr/local/lib/python3.5/dist-packages/bert_serving/client/__init__.py:286: UserWarning: some of your sentences have more tokens than "max_seq_len=150" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [12]:
# # train, test claim+evidence pair encode
# restart server with 
# bert-serving-start -model_dir /share/ShareFolder/uncased_L-24_H-1024_A-16/ -max_seq_len=150 -gpu_memory_fraction=0.9 -num_worker=1


train_pair_encode = bc.encode(list(train_df['claim_with_evi_text']))
np.save("./Sentence_encodings/train_pair_encode" + case, train_pair_encode)

# test_pair_encode = bc.encode(list(test_df['claim_with_evi_text']))
# np.save("./Sentence_encodings/test_pair_encode" + case, test_pair_encode)

### Load bert features from file

In [23]:

train_claim_features = np.load("./Sentence_encodings/train_claim_encode.npy")
# test_claim_features = np.load("./Sentence_encodings/test_claim_encode.npy")

train_evi_features = np.load("./Sentence_encodings/train_evi_encode.npy")
# test_evi_features = np.load("./Sentence_encodings/test_evi_encode.npy")

train_pair_features = np.load("./Sentence_encodings/train_pair_encode.npy")
# test_pair_features = np.load("././Sentence_encodings/test_pair_encode.npy")

train_claim_features_cased = np.load("./Sentence_encodings/train_claim_encode_cased.npy")
# test_claim_features_cased = np.load("./Sentence_encodings/test_claim_encode_cased.npy")

train_evi_features_cased = np.load("./Sentence_encodings/train_evi_encode_cased.npy")
# test_evi_features_cased = np.load("./Sentence_encodings/test_evi_encode_cased.npy")

train_pair_features_cased = np.load("./Sentence_encodings/train_pair_encode_cased.npy")
# test_pair_features_cased = np.load("././Sentence_encodings/test_pair_encode_cased.npy")


In [24]:
x_train = np.concatenate([train_claim_features, train_evi_features, train_pair_features, train_claim_features_cased, train_evi_features_cased, train_pair_features_cased], axis=1)
y_train = train_df[train_df.columns[0:3]].values
x_test = np.concatenate([test_claim_features, test_evi_features, test_pair_features, test_claim_features_cased, test_evi_features_cased, test_pair_features_cased], axis=1)

In [25]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(145449, 6144)
(145449, 3)
(5001, 6144)


## Simple MLP to train (with bert features)

In [7]:
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import Adam

seed = 7
np.random.seed(seed)


model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(units=50, activation='relu', input_dim=x_train.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(units=1, activation='sigmoid'))
# optimizer = Adam(lr=0.01)
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer='adam', metrics=['accuracy'])

model.summary()
# SVG(model_to_dot(model).create(prog='dot', format='svg'))

# callbacks
filepath="best_weights_sentence.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
earlyStopping = EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')

callbacks_list = [checkpoint, earlyStopping]

# model.fit(x=x_train, y=y_train, batch_size=32, epochs=50, validation_split=0.1, callbacks=callbacks_list)

NameError: name 'x_train' is not defined